In [ ]:
# Cell 0: Setup — clone repo, link data
import os, subprocess
REPO = "/kaggle/working/competition_package"
os.chdir("/kaggle/working")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.system("ln -sf /kaggle/input/wunderfund-predictorium/train.parquet datasets/train.parquet")
os.system("ln -sf /kaggle/input/wunderfund-predictorium/valid.parquet datasets/valid.parquet")
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print("Ready!")

In [ ]:
# Cell 1: TCN kill test — seeds 42-46 (pre-committed contiguous block)
# TCN config: 32 hidden channels, 6 dilated blocks, ~9K params, receptive field 127
# Kill rule: val < 0.2400 (50% of GRU baseline) = architecture fundamentally broken
# Worth including in ensemble if val > 0.2500 AND adds orthogonal diversity
import os
os.chdir("/kaggle/working/competition_package")

for seed in range(42, 47):
    print(f"\n{'='*60}")
    print(f"Training tcn_base_v1 seed {seed}")
    print(f"{'='*60}")
    os.system(
        f"python -u scripts/train.py "
        f"--config configs/tcn_base_v1.yaml "
        f"--seed {seed} --device cuda"
    )

print("\nAll 5 TCN seeds complete!")

In [ ]:
# Cell 2: Compare TCN val scores
import os, glob, torch
os.chdir("/kaggle/working/competition_package")

results = []
for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    score = ckpt.get("best_score", ckpt.get("val_score", None))
    name = os.path.basename(pt)
    if isinstance(score, (int, float)):
        results.append((name, float(score)))
    else:
        results.append((name, 0.0))

results.sort(key=lambda x: x[1], reverse=True)

print(f"{'Model':<55} {'Val Score':>10}")
print("-" * 67)
for name, score in results:
    print(f"{name:<55} {score:>10.4f}")

tcn_scores = [s for n, s in results if "tcn" in n]
if tcn_scores:
    mean_tcn = sum(tcn_scores) / len(tcn_scores)
    print(f"\nTCN mean: {mean_tcn:.4f} (GRU tw2 mean: 0.2613, p1 mean: 0.2627)")
    print(f"TCN best: {max(tcn_scores):.4f}")
    if mean_tcn > 0.2500:
        print(f"  VIABLE: TCN shows signal. Test ensemble diversity.")
    elif mean_tcn > 0.2400:
        print(f"  MARGINAL: TCN weak but might help via diversity.")
    else:
        print(f"  KILL: TCN architecture doesn't work on this data.")

In [ ]:
# Cell 3: Strip checkpoints + zip for download
import os, torch, glob, shutil
os.chdir("/kaggle/working/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
    }
    out = f"logs/slim/{os.path.basename(pt)}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{os.path.basename(pt)}: {orig:.1f}MB -> {new:.1f}MB")

for npz in sorted(glob.glob("logs/normalizer_*.npz")):
    shutil.copy(npz, f"logs/slim/{os.path.basename(npz)}")
    print(f"Copied {os.path.basename(npz)}")

shutil.make_archive("/kaggle/working/tcn_seeds", "zip",
                     "/kaggle/working/competition_package/logs/slim")
sz = os.path.getsize("/kaggle/working/tcn_seeds.zip") / 1e6
print(f"\ntcn_seeds.zip: {sz:.1f}MB")
print("Download from: /kaggle/working/tcn_seeds.zip")